In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

%matplotlib inline

# Read Data

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
# Fill NA values with the mean
train = train.fillna(train.mean())
test = test.fillna(train.mean())

In [8]:
# Split into X and y
X_train = train.drop(['id', 'Y'], axis=1)
y_train = train['Y']

X_test = test.drop(['id'], axis=1)

# Helper Functions

In [15]:
def to_file(filename, ids, preds):
    with open(filename, 'w') as f:
        f.write('id,Y\n')
        for num, pred in zip(ids, preds):
            f.write('{},{}\n'.format(num, pred))

In [9]:
def print_cv_results(model):
    print("Best parameter set found:\n")
    print(model.best_params_)
    print()
    for params, mean_score, scores in model.grid_scores_:
        print("{0:.3f} (+/-{1:.03f}) for {2}".format(mean_score, scores.std() * 2, params))
    print()

# TensorFlow

In [14]:
import skflow
from sklearn.model_selection import cross_val_score

params = {
    'hidden_units': [[5], [5, 3, 4]],
    'learning_rate': [0.05, 0.1],
}

dnn = GridSearchCV(skflow.TensorFlowDNNClassifier(hidden_units=[5], n_classes=2, steps=50000, verbose=0),
                   param_grid=params, cv=5, n_jobs=-1, scoring='roc_auc')
dnn.fit(X_train, y_train)

print_cv_results(dnn)

/Users/rohannagar/anaconda/lib/python3.5/site-packages/skflow/io/data_feeder.py:217: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out.itemset((i, self.y[sample]), 1.0)
/Users/rohannagar/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/Users/rohannagar/anaconda/lib/python3.5/site-packages/skflow/io/data_feeder.py:217: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out.itemset((i, self.y[sample]), 1.0)
/Users/rohannagar/anaconda/lib/python3.5/site-packages/skflow/io/data_feeder.py:217: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out.itemset((

Best parameter set found:

{'hidden_units': [5, 3, 4], 'learning_rate': 0.05}

0.501 (+/-0.001) for {'hidden_units': [5], 'learning_rate': 0.05}
0.500 (+/-0.001) for {'hidden_units': [5], 'learning_rate': 0.1}
0.503 (+/-0.004) for {'hidden_units': [5, 3, 4], 'learning_rate': 0.05}
0.502 (+/-0.003) for {'hidden_units': [5, 3, 4], 'learning_rate': 0.1}



Not very good scores with this data, but you can do some more parameter tuning to try and improve the scores. This was mainly to show you how to use a TensorFlow Neural Network classifier.

Let's train a model and predict on the test data for completion.

In [17]:
classifier = skflow.TensorFlowDNNClassifier(hidden_units=[5, 3, 4], n_classes=2, steps=50000, learning_rate=0.05)
classifier.fit(X_train, y_train)
preds = classifier.predict_proba(X_test)[:, 1]
to_file('tensorflow.csv', test['id'], preds)

Step #1, avg. loss: 0.63302
Step #5001, epoch #3, avg. loss: 0.27369
Step #10001, epoch #6, avg. loss: 0.24658
Step #15001, epoch #9, avg. loss: 0.24609
Step #20001, epoch #12, avg. loss: 0.24653
Step #25001, epoch #15, avg. loss: 0.24658
Step #30001, epoch #19, avg. loss: 0.24678
Step #35001, epoch #22, avg. loss: 0.24716
Step #40001, epoch #25, avg. loss: 0.24594
Step #45001, epoch #28, avg. loss: 0.24639


/Users/rohannagar/anaconda/lib/python3.5/site-packages/skflow/io/data_feeder.py:217: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  out.itemset((i, self.y[sample]), 1.0)
